## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [Colab](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/Module2_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---

In [1]:
!pip install yfinance

In [9]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import requests
import re

### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

---

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [4]:
ipos = ipo_dfs[0]
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [5]:
# convert 'Filing Date' to datetime
ipos['Filing Date'] = pd.to_datetime(ipos['Filing Date'])

In [6]:
# convert 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
ipos['Shares Offered'] = pd.to_numeric(ipos['Shares Offered'], errors='coerce')

In [13]:
ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           326 non-null    datetime64[ns]
 1   Symbol                326 non-null    object        
 2   Company Name          326 non-null    object        
 3   Price Range           326 non-null    object        
 4   Shares Offered        252 non-null    float64       
 5   Avg_price             326 non-null    object        
 6   Shares_offered_value  249 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 18.0+ KB


In [8]:
def extract_numbers(input_string):
    '''function to extract the 'Price Range' numbers and do the average calculation'''
    prices = re.findall(r'\d+\.\d+', input_string)
    if len(prices) == 0:
        return [np.nan]
    elif len(prices) == 1:
        return float(prices[0])
    else:
        return np.mean([float(price) for price in prices])

# Ejemplos
examples = ["$8.00 - $10.00", "$10.00"]
for example in examples:
    res = extract_numbers(example)
    print(f"{example} ==> {res}")

$8.00 - $10.00 ==> 9.0
$10.00 ==> 10.0


In [9]:
# define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified,
# to the price (if only one number is provided), or to the average of 2 prices (if a range is given)

ipos['Avg_price'] = ipos['Price Range'].apply(lambda x:extract_numbers(x))

In [11]:
# define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price"
# (when both columns are defined; otherwise, it's NaN)


def calculate_shares_offered_value(row):
    '''function to calculate Shares_offered_value'''
    if not pd.isnull(row['Shares Offered']) and not pd.isnull(row['Avg_price']):
        return row['Shares Offered'] * row['Avg_price']
    else:
        return np.nan

# applies the function along the rows of the DataFrame
ipos['Shares_offered_value'] = ipos.apply(lambda row: calculate_shares_offered_value(row), axis=1)

In [12]:
ipos.tail()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0,9.0,13500000.0
322,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0,10.0,50000000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0,9.0,22500000.0
324,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0,4.0,17200000.0
325,2018-12-27,FBOX,Fit Boxx Holdings Limited,$5.00 - $6.50,2000000.0,5.75,11500000.0


In [18]:
# Filter records for 2023 and Fridays
ipos_2023_fridays = ipos[(ipos['Filing Date'].dt.year == 2023) & (ipos['Filing Date'].dt.dayofweek == 4)]

In [17]:
ipos_2023_fridays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 49 to 166
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Avg_price             32 non-null     object        
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 2.0+ KB


In [19]:
# Add the value of the shares offered for these records
total_shares_offered_value = ipos_2023_fridays['Shares_offered_value'].sum()

# Convert the sum to millions of dollars and round to the nearest integer
total_shares_offered_value_millions = round(total_shares_offered_value / 1_000_000)

print("Total sum in $m for all filings during 2023 that happened on Fridays:", total_shares_offered_value_millions)


Total sum in $m for all filings during 2023 that happened on Fridays: 286


### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Addtional:
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


---

In [87]:
# get all OHLCV daily prices for all stocks

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [88]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [89]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [90]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [91]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [92]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$26.99,16.21%


In [93]:
# Need to convert everything to a proper type (date, str, int, float, etc.)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      218 non-null    object
 1   Symbol        218 non-null    object
 2   Company Name  218 non-null    object
 3   IPO Price     218 non-null    object
 4   Current       218 non-null    object
 5   Return        218 non-null    object
dtypes: object(6)
memory usage: 10.3+ KB


In [94]:
# convert to datetime
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')

In [95]:
# Problem --> not always the columns are filled
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [96]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

In [97]:
# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [98]:
# Correctly applied transformations with 'defensive' techniques, but now not all are non-null
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      218 non-null    datetime64[ns]
 1   Symbol        218 non-null    object        
 2   Company Name  218 non-null    object        
 3   IPO Price     218 non-null    float64       
 4   Current       218 non-null    float64       
 5   Return        217 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.3+ KB


In [99]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
stacked_ipos_df.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          1
dtype: int64

In [100]:
# Do you want to leave the record or not?
stacked_ipos_df[stacked_ipos_df.Return.isnull()]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
13,2024-04-17,JUNE,Junee Limited,4.0,4.0,NaN


In [101]:
# now you can operate with columns as a numeric type
stacked_ipos_df['IPO Price'].mean()

11.060229357798166

In [102]:
# generate a new field -- SIMPLE calculation (no function needed)
stacked_ipos_df['Price Increase'] = stacked_ipos_df['Current'] - stacked_ipos_df['IPO Price']

In [103]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,Price Increase
0,2024-05-01,VIK,Viking Holdings Ltd.,24.0,26.99,0.1621,2.99


In [104]:
# Descriptive Analytics of a dataset
stacked_ipos_df.describe()

,IPO Date,IPO Price,Current,Return,Price Increase
count,218,218.000000,218.000000,217.000000,218.000000
mean,2023-09-08 03:44:35.229357824,11.060229,11.069633,-0.201485,0.009404
min,2023-01-13 00:00:00,2.500000,0.000000,-0.999600,-23.280000
25%,2023-04-20 00:00:00,4.000000,1.242500,-0.730000,-3.667500
50%,2023-09-14 12:00:00,8.000000,5.715000,-0.235500,-1.770000
75%,2024-01-24 18:00:00,13.750000,10.840000,0.057000,0.517500
max,2024-05-01 00:00:00,92.000000,116.530000,4.517500,47.000000
std,NaN,11.245650,16.979522,0.703658,8.947612


In [106]:
# with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers
specific_date = pd.to_datetime('2024-03-01')
filter_ipos = stacked_ipos_df[stacked_ipos_df['IPO Date'] < specific_date]

In [107]:
filter_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 33 to 217
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   IPO Date        185 non-null    datetime64[ns]
 1   Symbol          185 non-null    object        
 2   Company Name    185 non-null    object        
 3   IPO Price       185 non-null    float64       
 4   Current         185 non-null    float64       
 5   Return          185 non-null    float64       
 6   Price Increase  185 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 11.6+ KB


In [108]:
# remove 'RYZB', as it is no longer available on Yahoo Finance.
filter_ipos = filter_ipos.drop(filter_ipos.loc[filter_ipos['Symbol'] == 'RYZB'].index)

In [109]:
# ERROR:yfinance:['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')
# remove 'PTHR'
filter_ipos = filter_ipos.drop(filter_ipos.loc[filter_ipos['Symbol'] == 'PTHR'].index)

In [110]:
filter_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 33 to 217
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   IPO Date        183 non-null    datetime64[ns]
 1   Symbol          183 non-null    object        
 2   Company Name    183 non-null    object        
 3   IPO Price       183 non-null    float64       
 4   Current         183 non-null    float64       
 5   Return          183 non-null    float64       
 6   Price Increase  183 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 11.4+ KB


In [111]:
filter_ipos.tail()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,Price Increase
213,2023-01-25,QSG,QuantaSing Group Ltd,12.5,3.16,-0.7416,-9.34
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",5.0,0.47,-0.9068,-4.53
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",15.0,35.77,1.3847,20.77
216,2023-01-13,ISRL,Israel Acquisitions Corp,10.0,10.94,0.0940,0.94
217,2023-01-13,MGOL,MGO Global Inc.,5.0,0.24,-0.9512,-4.76


In [112]:
# adjust the symbol name
filter_ipos['Symbol'] = filter_ipos['Symbol'].replace('IBAC', 'IBACU')

In [69]:
#download each stock has data on Yahoo Finance

def download_data_for_symbols(df):

    dfs = []
    for symbol in df['Symbol'].unique():
        # download Yahoo Finance data for the current symbol
        data = yf.download(tickers=symbol, period='max', interval='1d')
        data['Symbol'] = symbol
        # Add the downloaded DataFrame to the list
        dfs.append(data)

    # Concatenate all DataFrames into one
    combined_df = pd.concat(dfs, axis=0)

    return combined_df

filter_ipos_data = download_data_for_symbols(filter_ipos)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [113]:
filter_ipos_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36917 entries, 2024-02-27 to 2024-05-02
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Open       36917 non-null  float64       
 1   High       36917 non-null  float64       
 2   Low        36917 non-null  float64       
 3   Close      36917 non-null  float64       
 4   Adj Close  36917 non-null  float64       
 5   Volume     36917 non-null  int64         
 6   Symbol     36917 non-null  object        
 7   Date       36917 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 2.5+ MB


In [74]:
# to be used in joins
filter_ipos_data['Date'] = filter_ipos_data.index.date

In [75]:
filter_ipos_data.tail()

,Open,High,Low,Close,Adj Close,Volume,Symbol,Date
Date,,,,,,,,
2024-04-26,0.2250,0.2320,0.2160,0.2210,0.2210,31300,MGOL,2024-04-26
2024-04-29,0.2350,0.2480,0.2240,0.2400,0.2400,128300,MGOL,2024-04-29
2024-04-30,0.2410,0.2600,0.2220,0.2360,0.2360,202100,MGOL,2024-04-30
2024-05-01,0.2360,0.2520,0.2360,0.2500,0.2500,16000,MGOL,2024-05-01
2024-05-02,0.2425,0.2489,0.2277,0.2327,0.2327,115449,MGOL,2024-05-02


In [117]:
nvo_df = filter_ipos_data

In [119]:
nvo_df.tail()

,Open,High,Low,Close,Adj Close,Volume,Symbol,Date
Date,,,,,,,,
2024-04-26,0.2250,0.2320,0.2160,0.2210,0.2210,31300,MGOL,2024-04-26
2024-04-29,0.2350,0.2480,0.2240,0.2400,0.2400,128300,MGOL,2024-04-29
2024-04-30,0.2410,0.2600,0.2220,0.2360,0.2360,202100,MGOL,2024-04-30
2024-05-01,0.2360,0.2520,0.2360,0.2500,0.2500,16000,MGOL,2024-05-01
2024-05-02,0.2425,0.2489,0.2277,0.2327,0.2327,115449,MGOL,2024-05-02


### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS.

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

---


### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


---

### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

---

## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02

---